Imports and setup

In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import random

from keras import Sequential
from keras.layers import Conv2D, BatchNormalization, Activation, Dropout, MaxPooling2D, Dense, Flatten

Dataset imagenet for validation crossplatform (not needed at the moment but maybe helpfull later)

In [2]:
(ds_test), ds_info = tfds.load(
    'imagenet_v2',
    split=['test[:90%]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

NameError: name 'tfds' is not defined

Load Image

In [4]:
testtabby = load_img("../data/tabby-cat.png")
testtabby = testtabby.resize((299, 299))
testtabby = img_to_array(testtabby)
testtabby_input = np.expand_dims(testtabby, axis=0)
testtabby = keras.applications.xception.preprocess_input(testtabby_input[None])  

Normalize function (not needed at the moment)

In [5]:


def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(299)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)



NameError: name 'ds_test' is not defined

Model no / Monte Carlo

In [12]:
modelNoMC = tf.keras.applications.Xception(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

modelMC = tf.keras.applications.Xception(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

model = tf.keras.applications.Xception(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

#modelNoMC.summary()
#print(len(modelMC.layers))

Sort out conv Layers

In [10]:
#other logic only apply to dense
applyTo = []

denseList = ["Dense"]

#starts at one because dont wanna change input/output
#only layer dense is Output :D
for i in range(len(modelMC.layers)):
    if(modelMC.layers[i].__class__.__name__ in denseList):
        applyTo.append(i)

#print(applyTo)

#skip conv layers
skipLayers = []

convList = ["Conv1D", "Conv2D", "Conv3D", "SeparableConv1D", "SeparableConv2D", "DepthwiseConv2D", "Conv1DTranspose", "Conv2DTranspose", "Conv3DTranspose"]

#starts at one because dont wanna change input/output
for i in range(1, len(modelMC.layers)-1):
    if(modelMC.layers[i].__class__.__name__ in convList):
        skipLayers.append(i)

#print(skipLayers)

Weight Changing One by One trough Layers

In [11]:
dropout_rate = 90
loopyloops = 10

y_samples_readable = []
for p in range(loopyloops):
    for i in range(len(modelMC.layers)):
        if((i) in applyTo):
            b = []
            b = model.layers[i].get_weights()
            for q in range(len(b)):
                if(str(type(b[q])) == "<class 'numpy.float32'>"):
                    if(random.randint(1, 100) <= dropout_rate):
                        #print("1. change to zero")
                        b[q] = 0.00000000
                else:
                    for j in range(len(b[q])):
                        if(str(type(b[q][j])) == "<class 'numpy.float32'>"):
                            if(random.randint(1, 100) <= dropout_rate):
                                #print("2. change to zero")
                                b[q][j] = 0.00000000
                        else:
                            for k in range(len(b[q][j])):
                                if(str(type(b[q][j][k])) == "<class 'numpy.float32'>"):
                                    if(random.randint(1, 100) <= dropout_rate):
                                        #print("3. change to zero")
                                        b[q][j][k] = 0.00000000
                                else:
                                    for l in range(len(b[q][j][k])):
                                        if(str(type(b[q][j][k][l])) == "<class 'numpy.float32'>"):
                                            if(random.randint(1, 100) <= dropout_rate):
                                                #print("4. change to zero")
                                                b[q][j][k][l] = 0.00000000
                                        else:        
                                            for m in range(len(b[q][j][k][l])):
                                                if(str(type(b[q][j][k][l][m])) == "<class 'numpy.float32'>"):
                                                    if(random.randint(1, 100) <= dropout_rate):
                                                        #print("5. change to zero")
                                                        b[q][j][k][l][m] = 0.00000000
                                                else:
                                                    for n in range(len(b[q][j][k][l][m])):
                                                        if(str(type(b[q][j][k][l][m][n])) == "<class 'numpy.float32'>"):
                                                            if(random.randint(1, 100) <= dropout_rate):
                                                                #print("6. change to zero")
                                                                b[q][j][k][l][m][n] = 0.00000000
                                                        else:
                                                            for o in range(len(b[q][j][k][l][m][n])):
                                                                if(str(type(b[q][j][k][l][m][n][o])) == "<class 'numpy.float32'>"):
                                                                    if(random.randint(1, 100) <= dropout_rate):
                                                                        #print("7. change to zero")
                                                                        b[q][j][k][l][m][n][o] = 0.00000000
                modelMC.layers[i].set_weights(b)

    #weight adjustment
    #WEIGHTS = modelMC.get_weights()
    #WEIGHTS[0] *= 1/(dropout_rate/100)
    #modelMC.set_weights(WEIGHTS)

    #compile
    #modelMC.compile(
    #optimizer='Adam',
    #loss='sparse_categorical_crossentropy',
    #metrics=['accuracy']
    #)

    predictions = modelMC.predict(testtabby_input)
    y_samples_readable.append(keras.applications.xception.decode_predictions(predictions))
    #print(keras.applications.xception.decode_predictions(predictions))



counter = 0
predictionNormalModel = keras.applications.xception.decode_predictions(modelNoMC.predict(testtabby_input))
for k in range(len(y_samples_readable)):
    print(y_samples_readable[k])
    if (y_samples_readable[k][0][0][1] == predictionNormalModel[0][0][1]):
        counter = counter + 1

print (predictionNormalModel)
print (str(counter/len(y_samples_readable)))
                                                    
    
        



KeyboardInterrupt



Random Weight changing 

In [ ]:
#hopefull try
dropout_rate = 10
loopyloops = 10

y_samples_readable = []
for p in range(loopyloops):
    a = []
    a = model.get_weights()
    for i in range(1, len(a)-1):
        if((i) not in skipLayers or True):
            if(str(type(a[i])) == "<class 'numpy.float32'>"):
                if(random.randint(1, 100) <= dropout_rate):
                    #print("1. change to zero")
                    a[i] = 0.00000000
            else:
                for j in range(len(a[i])):
                    if(str(type(a[i][j])) == "<class 'numpy.float32'>"):
                        if(random.randint(1, 100) <= dropout_rate):
                            #print("2. change to zero")
                            a[i][j] = 0.00000000
                    else:
                        for k in range(len(a[i][j])):
                            if(str(type(a[i][j][k])) == "<class 'numpy.float32'>"):
                                if(random.randint(1, 100) <= dropout_rate):
                                    #print("3. change to zero")
                                    a[i][j][k] = 0.00000000
                            else:
                                for l in range(len(a[i][j][k])):
                                    if(str(type(a[i][j][k][l])) == "<class 'numpy.float32'>"):
                                        if(random.randint(1, 100) <= dropout_rate):
                                            #print("4. change to zero")
                                            a[i][j][k][l] = 0.00000000
                                    else:        
                                        for m in range(len(a[i][j][k][l])):
                                            if(str(type(a[i][j][k][l][m])) == "<class 'numpy.float32'>"):
                                                if(random.randint(1, 100) <= dropout_rate):
                                                    #print("5. change to zero")
                                                    a[i][j][k][l][m] = 0.00000000
                                            else:
                                                for n in range(len(a[i][j][k][l][m])):
                                                    if(str(type(a[i][j][k][l][m][n])) == "<class 'numpy.float32'>"):
                                                        if(random.randint(1, 100) <= dropout_rate):
                                                            #print("6. change to zero")
                                                            a[i][j][k][l][m][n] = 0.00000000
                                                    else:
                                                        for o in range(len(a[i][j][k][l][m][n])):
                                                            if(str(type(a[i][j][k][l][m][n][o])) == "<class 'numpy.float32'>"):
                                                                if(random.randint(1, 100) <= dropout_rate):
                                                                    #print("7. change to zero")
                                                                    a[i][j][k][l][m][n][o] = 0.00000000
    modelMC.set_weights(a)

    #should be done for all weights
    #weight adjustment
    WEIGHTS = modelMC.get_weights()
    WEIGHTS[0] *= 1/(dropout_rate/100)
    modelMC.set_weights(WEIGHTS)

    #compile
    modelMC.compile(
    optimizer='Adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
    )

    predictions = modelMC.predict(testtabby_input)
    y_samples_readable.append(keras.applications.xception.decode_predictions(predictions))
    #print(keras.applications.xception.decode_predictions(predictions))



counter = 0
predictionNormalModel = keras.applications.xception.decode_predictions(modelNoMC.predict(testtabby_input))
for k in range(len(y_samples_readable)):
    print(y_samples_readable[k])
    if (y_samples_readable[k][0][0][1] == predictionNormalModel[0][0][1]):
        counter = counter + 1

print (predictionNormalModel)
print (str(counter/len(y_samples_readable)))
                                                    

In [25]:
print(a)

[array([[[[ 4.01871502e-01,  6.77069500e-02, -9.92160812e-02,
          -2.40881741e-01,  4.94409263e-01, -1.35092044e+00,
          -7.53455386e-02, -1.15802072e-01,  2.54089713e-01,
           4.12381589e-01, -2.02945337e-01, -5.65736771e-01,
           1.53094187e-01, -5.43646663e-02,  9.78396833e-02,
          -2.72699267e-01, -9.39905122e-02,  3.93782228e-01,
           3.33525687e-01,  2.95914084e-01, -3.14331800e-02,
          -1.99261308e-01,  1.59772888e-01, -3.25841278e-01,
          -1.50597319e-01, -7.27500841e-02,  1.80662051e-02,
           3.05232316e-01,  8.96351188e-02,  2.89342284e-01,
          -2.50950217e-01, -7.65512884e-01],
         [ 4.84812856e-02,  4.23621461e-02,  4.00314759e-03,
          -3.78118306e-02, -5.52299500e-01,  7.35755444e-01,
          -3.38940531e-01, -1.09324297e-02, -8.63540769e-02,
          -7.23446161e-02, -5.50143421e-01, -5.00105917e-01,
           2.29335129e-01,  2.17402652e-01, -1.51911721e-01,
          -3.03186446e-01,  1.59955341e

models Compile

In [26]:
modelNoMC.compile(
optimizer='Adam',
loss='sparse_categorical_crossentropy',
metrics=['accuracy']
)

modelMC.compile(
optimizer='Adam',
loss='sparse_categorical_crossentropy',
metrics=['accuracy']
)

predict

In [27]:
predictions = modelNoMC.predict(testtabby_input)
predictions = keras.applications.xception.decode_predictions(predictions)
print(predictions)

print("With MCD")

predictions = modelMC.predict(testtabby_input)
predictions = keras.applications.xception.decode_predictions(predictions)
print(predictions)

1/1 [==============================] - 3s 3s/step
[[('n02123045', 'tabby', 0.6505307), ('n02123159', 'tiger_cat', 0.23030522), ('n02124075', 'Egyptian_cat', 0.01496593), ('n02127052', 'lynx', 0.0042105187), ('n02123394', 'Persian_cat', 0.0010178831)]]
With MCD
1/1 [==============================] - 3s 3s/step
[[('n04428191', 'thresher', 0.9341652), ('n03980874', 'poncho', 0.04589951), ('n03444034', 'go-kart', 0.007156095), ('n03496892', 'harvester', 0.0066612125), ('n04604644', 'worm_fence', 0.005810678)]]
